### Modules

In [ ]:
# basic
import os, sys, glob, umap, pybedtools
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from liftover import get_lifter
from sklearn.metrics import confusion_matrix, roc_curve, precision_recall_curve, auc, silhouette_samples
from matplotlib_venn import venn2, venn3
from matplotlib.lines import Line2D
from Bio.Seq import IUPACData
from itertools import product

### Figure 3A

In [ ]:
def roc_val(pred_table, col):
    
    y_pred = pred_table[col]
    y_true = pred_table['gt']

    fpr, tpr, thersholds = roc_curve(y_true, y_pred, pos_label = 1)
    roc_auc = auc(fpr, tpr)

    out = [fpr, tpr, roc_auc]
    
    return out

def extend_ambiguous(seq):
    
    d = IUPACData.ambiguous_dna_values
    return list(map(''.join, product(*map(d.get, seq)))) 

In [ ]:
### benchmark with IVT-20%
# m6ATM
pred_pos = pd.read_csv('../data/ivtmix20_valid.csv', index_col = 0)
pred_pos['site'] = [x+'_'+str(y)+'_p' for x, y in zip(pred_pos['transcript'], pred_pos['position'])]

pred_neg = pd.read_csv('../data/ivtmix0_valid.csv', index_col = 0)
pred_neg['site'] = [x+'_'+str(y)+'_n' for x, y in zip(pred_neg['transcript'], pred_neg['position'])]

pred_table = pd.concat([pred_pos, pred_neg], axis = 0)
pred_table['gt'] = [1]*pred_pos.shape[0]+[0]*pred_neg.shape[0]

# m6Anet
pred_m6anet_pos = pd.read_csv('../data/ivtmix20_m6anet.csv')
pred_m6anet_pos['predicted'] = [1 if i>=0.5 else 0 for i in pred_m6anet_pos['probability_modified'] ]
pred_m6anet_pos['site'] = [x+'_'+str(y)+'_p' for x, y in zip(pred_m6anet_pos['transcript_id'], pred_m6anet_pos['transcript_position'])]

pred_m6anet_neg = pd.read_csv('../data/ivtmix0_m6anet.csv')
pred_m6anet_neg['site'] = [x+'_'+str(y)+'_n' for x, y in zip(pred_m6anet_neg['transcript_id'], pred_m6anet_neg['transcript_position'])]

pred_m6anet = pd.concat([pred_m6anet_pos, pred_m6anet_neg], axis = 0)
pred_m6anet['gt'] = [1]*pred_m6anet_pos.shape[0]+[0]*pred_m6anet_neg.shape[0]

# m6Abasecaller
pred_m6abase_pos = pd.read_csv('../data/ivtmix20_m6abase.tsv', sep = '\t')
pred_m6abase_pos.columns = ['chr', 'pos', 'ref_base', 'strand', 'mod', 'coverage', 'base_accuracy', 'ratio', 'probability']
pred_m6abase_pos['site'] = [x+'_'+str(y-1)+'_p' for x, y in zip(pred_m6abase_pos['chr'], pred_m6abase_pos['pos'])]
pred_m6abase_pos = pred_m6abase_pos[pred_m6abase_pos.site.isin(pred_pos.site.tolist())]
pred_m6abase_pos['probability'] = pred_m6abase_pos['probability'].replace(np.nan, 0)

pred_m6abase_neg = pd.read_csv('../data/ivtmix0_m6abase.tsv', sep = '\t')
pred_m6abase_neg.columns = ['chr', 'pos', 'ref_base', 'strand', 'mod', 'coverage', 'base_accuracy', 'ratio', 'probability']
pred_m6abase_neg['site'] = [x+'_'+str(y-1)+'_n' for x, y in zip(pred_m6abase_neg['chr'], pred_m6abase_neg['pos'])]
pred_m6abase_neg = pred_m6abase_neg[pred_m6abase_neg.site.isin(pred_neg.site.tolist())]
pred_m6abase_neg['probability'] = pred_m6abase_neg['probability'].replace(np.nan, 0)

pred_m6abase = pd.concat([pred_m6abase_pos, pred_m6abase_neg], axis = 0)
pred_m6abase['gt'] = [1]*pred_m6abase_pos.shape[0]+[0]*pred_m6abase_neg.shape[0]

# EpiNano
pred_epinano_pos = pd.read_csv('../data/ivtmix20_epinano.csv')
pred_epinano_pos = pred_epinano_pos.loc[:,['#Kmer', 'Window', 'Ref', 'Strand', 'Coverage', 'prediction', 'ProbM']]
pred_epinano_pos['site'] = [x+'_'+str(int(y.split('-')[0])+1)+'_p' for x, y in zip(pred_epinano_pos['Ref'], pred_epinano_pos['Window'])]
pred_epinano_pos = pred_epinano_pos[pred_epinano_pos.site.isin(pred_pos.site.tolist())]

pred_epinano_neg = pd.read_csv('../data/ivtmix0_epinano.csv')
pred_epinano_neg = pred_epinano_neg.loc[:,['#Kmer', 'Window', 'Ref', 'Strand', 'Coverage', 'prediction', 'ProbM']]
pred_epinano_neg['site'] = [x+'_'+str(int(y.split('-')[0])+1)+'_n' for x, y in zip(pred_epinano_neg['Ref'], pred_epinano_neg['Window'])]
pred_epinano_neg = pred_epinano_neg[pred_epinano_neg.site.isin(pred_neg.site.tolist())]

pred_epinano = pd.concat([pred_epinano_pos, pred_epinano_neg], axis = 0)
pred_epinano['gt'] = [1]*pred_epinano_pos.shape[0]+[0]*pred_epinano_neg.shape[0]

# Tombo
pred_tombo_pos = pd.read_csv('../data/ivtmix20_tombo.bed', sep = '\t', header = None)
pred_tombo_pos.columns = ['chr', 'start', 'end', 'id', 'probability']
pred_tombo_pos['site'] = [x+'_'+str(y)+'_p' for x, y in zip(pred_tombo_pos['chr'], pred_tombo_pos['start'])]

pred_tombo_neg = pd.read_csv('../data/ivtmix0_tombo.bed', sep = '\t', header = None)
pred_tombo_neg.columns = ['chr', 'start', 'end', 'id', 'probability']
pred_tombo_neg['site'] = [x+'_'+str(y)+'_n' for x, y in zip(pred_tombo_neg['chr'], pred_tombo_neg['start'])]

pred_tombo = pd.concat([pred_tombo_pos, pred_tombo_neg], axis = 0)
pred_tombo['gt'] = [1]*pred_tombo_pos.shape[0]+[0]*pred_tombo_neg.shape[0]

# MINES
pred_mines_pos = pd.read_csv('../data/ivtmix20_mines.bed', sep = '\t', header = None)
pred_mines_pos.columns = ['chr', 'start', 'end', 'motif', 'id', 'strand', 'probability', 'coverage']
pred_mines_pos['site'] = [x+'_'+str(y-1)+'_p' for x, y in zip(pred_mines_pos['chr'], pred_mines_pos['start'])]

pred_mines_neg = pd.read_csv('../data/ivtmix0_mines.bed', sep = '\t', header = None)
pred_mines_neg.columns = ['chr', 'start', 'end', 'motif', 'id', 'strand', 'probability', 'coverage']
pred_mines_neg['site'] = [x+'_'+str(y-1)+'_n' for x, y in zip(pred_mines_neg['chr'], pred_mines_neg['start'])]

pred_mines = pd.concat([pred_mines_pos, pred_mines_neg], axis = 0)
pred_mines['gt'] = [1]*pred_mines_pos.shape[0]+[0]*pred_mines_neg.shape[0]

# sites
sites = set.intersection(set(pred_table['site']),
                         set(pred_m6anet['site']),
                         set(pred_m6abase['site']),
                         set(pred_epinano['site']),
                         set(pred_tombo['site']),
                         set(pred_mines['site']))

pred_table = pred_table[pred_table.site.isin(sites)]
pred_m6anet = pred_m6anet[pred_m6anet.site.isin(sites)]
pred_m6abase = pred_m6abase[pred_m6abase.site.isin(sites)]
pred_epinano = pred_epinano[pred_epinano.site.isin(sites)]
pred_tombo = pred_tombo[pred_tombo.site.isin(sites)]
pred_mines = pred_mines[pred_mines.site.isin(sites)]

In [ ]:
### roc plot
# auc roc
roc_m6atm = roc_val(pred_table, 'probability')
roc_m6anet = roc_val(pred_m6anet, 'probability_modified')
roc_m6abase = roc_val(pred_m6abase, 'probability')
roc_epinano = roc_val(pred_epinano, 'ProbM')
roc_tombo = roc_val(pred_tombo, 'probability')
roc_mines = roc_val(pred_mines, 'probability')

# plot
sns.set_theme(style = 'white') # theme
tab_color = sns.color_palette('Set2')
fig, ax = plt.subplots(figsize = (8, 6)) # figure size

plt.plot(roc_m6atm[0], roc_m6atm[1], color = tab_color[0], lw = 2, label = 'm6ATM (area = {0:.3f})'.format(roc_m6atm[2]))
plt.plot(roc_m6anet[0], roc_m6anet[1], color = tab_color[1], lw = 2, label = 'm6Anet (area = {0:.3f})'.format(roc_m6anet[2]))
plt.plot(roc_m6abase[0], roc_m6abase[1], color = tab_color[2], lw = 2, label = 'm6ABasecaller (area = {0:.3f})'.format(roc_m6abase[2]))
plt.plot(roc_epinano[0], roc_epinano[1], color = tab_color[3], lw = 2, label = 'EpiNano (area = {0:.3f})'.format(roc_epinano[2]))
plt.plot(roc_tombo[0], roc_tombo[1], color = tab_color[4], lw = 2, label = 'Tombo (area = {0:.3f})'.format(roc_tombo[2]))
plt.plot(roc_mines[0], roc_mines[1], color = tab_color[5], lw = 2, label = 'MINES (area = {0:.3f})'.format(roc_mines[2]))

plt.plot([0, 1], [0, 1], color = 'gray', lw = 2, linestyle = '--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel('False Positive Rate', fontsize = 25)
plt.ylabel('True Positive Rate', fontsize = 25)  
plt.legend(loc = 'lower right', fontsize = 12)

plt.tick_params(labelsize = 16)

### Figure 3B

In [ ]:
def add_kmer(row):
    
    tx = row['transcript']
    pos = row['position']-1
    motif = ref_dict_tx[tx][pos-2:pos+3]
    motif = ''.join(motif)
    
    return motif

def pr_val(pred_table, col):
    
    y_pred = pred_table[col]
    y_true = pred_table['gt']

    precision, recall, thresholds = precision_recall_curve(y_true, y_pred, pos_label = 1)
    pr_auc = auc(recall, precision)

    out = [precision, recall, pr_auc]
    
    return out

#### HEK293

In [ ]:
### benchmark with HEK293
### here we use prediction results in genome coordination 
### for coordination conversion from transcript to genome, please check "tx_conversion.ipynb"

In [ ]:
### ground-truth m6A sites from m6A-SAC-seq
gt_table = pd.read_csv('../data/gt_sac_hek293.csv', index_col = 0)
gt_sites = sorted([x+'_'+str(y) for x,y in zip(gt_table['chr'], gt_table['center'])])

# remove m6A sites also reported by miCLIP-seq and keep the pool of m6A-negative sites more confident
gt_table_u = pd.read_csv('../data/gt_usites_hek293.csv', index_col = 0)
gt_sites_u = sorted([x+'_'+str(y) for x,y in zip(gt_table_u['chr'], gt_table_u['center'])])
coverage = 50

### prediction results
# m6ATM
pred = pd.read_csv('../data/hek293_m6atm.csv', index_col = 0)
pred = pred[pred.coverage>=coverage]
pred['gt'] = pred['gn_site'].isin(gt_sites)

# m6Anet
pred_m6anet = pd.read_csv('../data/hek293_m6anet.csv')
pred_m6anet = pred_m6anet[pred_m6anet.coverage>=coverage]
pred_m6anet['gt'] = pred_m6anet['gn_site'].isin(gt_sites)

# m6ABasecaller
pred_m6abase = pd.read_csv('../data/hek293_m6abase.csv')
pred_m6abase = pred_m6abase[pred_m6abase.coverage>=coverage]
pred_m6abase['gt'] = pred_m6abase['gn_site'].isin(gt_sites)

# Tombo
pred_tombo = pd.read_csv('../data/hek293_tombo.csv')
pred_tombo = pred_tombo[pred_tombo.coverage>=coverage]
pred_tombo['gt'] = pred_tombo['gn_site'].isin(gt_sites)

# MINES
pred_mines = pd.read_csv('../data/hek293_mines.csv')
pred_mines = pred_mines[pred_mines.coverage>=coverage]
pred_mines['gt'] = pred_mines['gn_site'].isin(gt_sites)

# sites intersection
sites = set.intersection(set(pred['gn_site']), set(pred_m6anet['gn_site']), set(pred_m6abase['gn_site']),
                         set(pred_tombo['gn_site']), set(pred_mines['gn_site']))

pred = pred[pred.gn_site.isin(sites)]

### roc and pr values
# m6ATM
pred_pos = pred[pred['gt'] == True]
pred_gt_n = pred[pred['gt'] == False]
pred_gt_not_u = pred_gt_n[~pred_gt_n['gn_site'].isin(gt_sites_u)] # pool for negative samples
pred_neg = pred_gt_not_u.sample(n = pred[pred['gt'] == True].shape[0], random_state = 0)

pred_all = pd.concat([pred_pos, pred_neg], axis = 0)

roc_m6atm = roc_val(pred_all, 'probability')
pr_m6atm = pr_val(pred_all, 'probability')

sites = pred_all['gn_site'].tolist()

### m6Anet
pred_m6anet_all = pred_m6anet[pred_m6anet.gn_site.isin(sites)]
roc_m6anet = roc_val(pred_m6anet_all, 'probability')
pr_m6anet = pr_val(pred_m6anet_all, 'probability')

### m6ABasecaller
pred_m6abase_all = pred_m6abase[pred_m6abase.gn_site.isin(sites)]
roc_m6abase = roc_val(pred_m6abase_all, 'probability')
pr_m6abase = pr_val(pred_m6abase_all, 'probability')

### Tombo
pred_tombo_all = pred_tombo[pred_tombo.gn_site.isin(sites)]
roc_tombo = roc_val(pred_tombo_all, 'probability')
pr_tombo = pr_val(pred_tombo_all, 'probability')

### MINES
pred_mines_all = pred_mines[pred_mines.gn_site.isin(sites)]
roc_mines = roc_val(pred_mines_all, 'probability')
pr_mines = pr_val(pred_mines_all, 'probability')

In [ ]:
### roc plot
# plot
sns.set_theme(style = 'white') # theme
tab_color = sns.color_palette('Set2')
fig, ax = plt.subplots(figsize = (8, 6)) # figure size

plt.plot(roc_m6atm[0], roc_m6atm[1], color = tab_color[0], lw = 2, label = 'm6ATM (area = {0:.3f})'.format(roc_m6atm[2]))
plt.plot(roc_m6anet[0], roc_m6anet[1], color = tab_color[1], lw = 2, label = 'm6Anet (area = {0:.3f})'.format(roc_m6anet[2]))
plt.plot(roc_m6abase[0], roc_m6abase[1], color = tab_color[2], lw = 2, label = 'm6ABasecaller (area = {0:.3f})'.format(roc_m6abase[2]))
plt.plot(roc_tombo[0], roc_tombo[1], color = tab_color[4], lw = 2, label = 'Tombo (area = {0:.3f})'.format(roc_tombo[2]))
plt.plot(roc_mines[0], roc_mines[1], color = tab_color[5], lw = 2, label = 'MINES (area = {0:.3f})'.format(roc_mines[2]))

plt.plot([0, 1], [0, 1], color = 'gray', lw = 2, linestyle = '--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel('False Positive Rate', fontsize = 25)
plt.ylabel('True Positive Rate', fontsize = 25)  
plt.legend(loc = 'lower right', fontsize = 12)

plt.tick_params(labelsize = 16)

### Figure S6

In [ ]:
### pr plot
# plot
sns.set_theme(style = 'white') # theme
tab_color = sns.color_palette('Set2')
fig, ax = plt.subplots(figsize = (8, 6)) # figure size

plt.plot(pr_m6atm[1], pr_m6atm[0], color = tab_color[0], lw = 2, label = 'm6ATM (area = {0:.3f})'.format(pr_m6atm[2]))
plt.plot(pr_m6anet[1], pr_m6anet[0], color = tab_color[1], lw = 2, label = 'm6Anet (area = {0:.3f})'.format(pr_m6anet[2]))
plt.plot(pr_m6abase[1], pr_m6abase[0], color = tab_color[2], lw = 2, label = 'm6ABasecaller (area = {0:.3f})'.format(pr_m6abase[2]))
plt.plot(pr_tombo[1], pr_tombo[0], color = tab_color[4], lw = 2, label = 'Tombo (area = {0:.3f})'.format(pr_tombo[2]))
plt.plot(pr_mines[1], pr_mines[0], color = tab_color[5], lw = 2, label = 'MINES (area = {0:.3f})'.format(pr_mines[2]))

plt.plot([1, 0], [0, 1], color = 'gray', lw = 2, linestyle = '--')
plt.xlim([-0.01, 1.0])
plt.ylim([0.0, 1.05])

plt.xlabel('Recall', fontsize = 25)
plt.ylabel('Precision', fontsize = 25)  
plt.legend(loc = 'lower right', fontsize = 12)

plt.tick_params(labelsize = 16)

In [ ]:
### repeat 10 times negative sample selection
roc_m6atm_list = []
pr_m6atm_list = []
roc_m6anet_list = []
pr_m6anet_list = []
roc_m6abase_list = []
pr_m6abase_list = []
roc_tombo_list = []
pr_tombo_list = []
roc_mines_list = []
pr_mines_list = []
for i in range(10):

    ### m6atm
    # sites not in gt_m6A sites
    pred_pos = pred[pred['gt'] == True]

    pred_gt_n = pred[pred['gt'] == False]
    pred_gt_not_u = pred_gt_n[~pred_gt_n['gn_site'].isin(gt_sites_u)]
    pred_neg = pred_gt_not_u.sample(n = pred[pred['gt'] == True].shape[0], random_state = i)

    pred_all = pd.concat([pred_pos, pred_neg], axis = 0)

    roc_m6atm_list.append(roc_val(pred_all, 'probability'))
    pr_m6atm_list.append(pr_val(pred_all, 'probability'))

    sites = pred_all['gn_site'].tolist()

    ### m6anet
    pred_m6anet_all = pred_m6anet[pred_m6anet.gn_site.isin(sites)]
    roc_m6anet_list.append(roc_val(pred_m6anet_all, 'probability'))
    pr_m6anet_list.append(pr_val(pred_m6anet_all, 'probability'))

    ### m6abase
    pred_m6abase_all = pred_m6abase[pred_m6abase.gn_site.isin(sites)]
    roc_m6abase_list.append(roc_val(pred_m6abase_all, 'probability'))
    pr_m6abase_list.append(pr_val(pred_m6abase_all, 'probability'))

    ### tombo
    pred_tombo_all = pred_tombo[pred_tombo.gn_site.isin(sites)]
    roc_tombo_list.append(roc_val(pred_tombo_all, 'probability'))
    pr_tombo_list.append(pr_val(pred_tombo_all, 'probability'))

    ### mines
    pred_mines_all = pred_mines[pred_mines.gn_site.isin(sites)]
    roc_mines_list.append(roc_val(pred_mines_all, 'probability'))
    pr_mines_list.append(pr_val(pred_mines_all, 'probability'))
    
roc_m6atm = np.array([i[2] for i in roc_m6atm_list]).mean()
roc_m6anet = np.array([i[2] for i in roc_m6anet_list]).mean()
roc_m6abase = np.array([i[2] for i in roc_m6abase_list]).mean()
roc_tombo = np.array([i[2] for i in roc_tombo_list]).mean()
roc_mines = np.array([i[2] for i in roc_mines_list]).mean()

pr_m6atm = np.array([i[2] for i in pr_m6atm_list]).mean()
pr_m6anet = np.array([i[2] for i in pr_m6anet_list]).mean()
pr_m6abase = np.array([i[2] for i in pr_m6abase_list]).mean()
pr_tombo = np.array([i[2] for i in pr_tombo_list]).mean()
pr_mines = np.array([i[2] for i in pr_mines_list]).mean()

In [ ]:
### bar plot 
# plot
sns.set_theme(style = 'white') # theme
tab_color = sns.color_palette() # color palette
fig, ax = plt.subplots(figsize = (8, 6)) # figure size

x = np.array(['m6ATM', 'm6Anet', 'm6ABasecaller', 'Tombo', 'MINES'])  # the label locations
roc_vals = [roc_m6atm, roc_m6anet, roc_m6abase, roc_tombo, roc_mines]
pr_vals = [pr_m6atm, pr_m6anet, pr_m6abase, pr_tombo, pr_mines]

line1 = plt.plot(x, roc_vals, color = tab_color[8], alpha = 0.8, lw = 5, marker = 'o', markersize = 8, label = 'AUC_ROC')
line2 = plt.plot(x, pr_vals, color = tab_color[9], alpha = 0.8, lw = 5, marker = 'o', markersize = 8, label = 'AUC_PR')

plt.xlim([-0.3, 4.3])
plt.ylim([0.46, 0.95])
plt.legend(loc = 'upper right', fontsize = 12)

ax.tick_params(labelsize = 14)
sns.despine()


### Figure 3C

In [ ]:
### m6ATM
pred_m6a_gn = pd.read_csv('../data/hek293_m6atm.csv', index_col = 0)
pred_m6a_gn = pred_m6a_gn[pred_m6a_gn['m6a'].str.contains('yes')]
pred_m6a_gn['gn_site0'] = [x+'_'+str(y) for x, y in zip(pred_m6a_gn['chrom'], pred_m6a_gn['gn_pos'])]

In [ ]:
### m6A-SAC-seq and miLCIP-seq
sac_gt = pd.read_csv('../data/gt_sac_hek293.csv', index_col = 0)
miclip_gt = pd.read_csv('../data/gt_cims_hek293.csv', index_col = 0)

In [ ]:
### sets
set1 = set(pred_m6a_gn.gn_site0.tolist())
set2 = set(sac_gt.gn_site.tolist())
set3 = set(miclip_gt.gn_site.tolist())

### venn3
sns.set_theme() # theme
fig, ax = plt.subplots(figsize = (8, 8)) # figure size
plt.gca().set_facecolor('white') # background
tab_color = sns.color_palette('Set2')

venn = venn3([set1, set2, set3], ('m6ATM', 'm6A-SAC-seq', 'miCLIP'), set_colors = (tab_color[0], tab_color[1], tab_color[2]))

### Figure S12

In [ ]:
### All DRACH sites profile
pred_gn = pd.read_csv('../data/hek293_m6atm.csv', index_col = 0)
pred_gn['gn_site0'] = [x+'_'+str(y) for x, y in zip(pred_gn['chrom'], pred_gn['gn_pos'])]
sites_available = list(set(pred_gn.gn_site0))

# lost sites due to read coverage
sac_gt_lost = sac_gt.loc[~sac_gt.gn_site.isin(sites_available),:]
miclip_gt_lost = miclip_gt.loc[~miclip_gt.gn_site.isin(sites_available),:]

ratio_lost_sac = len(set(sac_gt_lost.gn_site))/len(set(sac_gt.gn_site))
ratio_lost_miclip = len(set(miclip_gt_lost.gn_site))/len(set(miclip_gt.gn_site))

In [ ]:
### bar plot 
# plot
sns.set_theme(style = 'white') # theme
tab_color = sns.color_palette() # color palette
fig, ax = plt.subplots(figsize = (10, 3)) # figure size

rows = ['miCLIP-seq', 'm6A-SAC-seq']
val1 = [1-ratio_lost_miclip, 1-ratio_lost_sac]
val2 = [ratio_lost_miclip, ratio_lost_sac]

rect1 = ax.barh(rows, val1, align = 'center', height = 0.8, color = tab_color[0], label = 'val1')
rect2 = ax.barh(rows, val2, align = 'center', left = val1, height = 0.8, color = tab_color[1], label = 'val2')

ax.bar_label(rect1, fmt = '%.2f', label_type = 'center', fontsize = 18, weight = 'bold', color = 'white')
ax.bar_label(rect2, fmt = '%.2f', label_type = 'center', fontsize = 18, weight = 'bold', color = 'white')

plt.legend(labels  = ['Sites above the detection threshold of m6ATM (read coverage >= 20)', 'Sites below the detection threshold of m6ATM (read coverage < 20)'],
           fontsize = 16, bbox_to_anchor = (0.96, 1))

ax.invert_yaxis()
ax.set_xticks([])
ax.tick_params(labelsize = 20)
sns.despine()